In [132]:
import pandas as pd
import fastf1 as ff1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re

In [133]:
# Enable FastF1 cache to avoid re-downloading
import os
cache_dir = 'f1_cache'
os.makedirs(cache_dir, exist_ok=True)
ff1.Cache.enable_cache(cache_dir)
print(f'FastF1 cache enabled: {cache_dir}')

FastF1 cache enabled: f1_cache


In [134]:
cycle1 = list(range(2017, 2022))
cycle2 = list(range(2022, 2026))

## Scraping GP data\n
Returns: `[(year, round, gp_name), ...]`

In [135]:
def scrape_f1_races(years):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    all_races = []
    
    for year in years:
        print(f'Scraping {year}...')
        url = f'https://www.formula1.com/en/racing/{year}'
        driver = webdriver.Chrome(options=options)
        year_races = []
        
        try:
            driver.get(url)
            time.sleep(3)
            
            try:
                cookie_btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.trustarc-agree-btn, button[id*='accept']"))
                )
                cookie_btn.click()
                time.sleep(1)
            except:
                pass
            
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(2)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            race_links = soup.find_all('a', href=lambda x: x and f'/racing/{year}/' in str(x))
            
            seen_rounds = set()
            for link in race_links:
                text = link.get_text(separator=' ', strip=True)
                if 'testing' in text.lower():
                    continue
                
                # Fixed regex - use raw string
                round_match = re.search("ROUND\s+(\d+)", text)
                if not round_match:
                    continue
                
                round_num = int(round_match.group(1))
                if round_num in seen_rounds:
                    continue
                
                href = link.get('href', '')
                gp_slug = href.rstrip('/').split('/')[-1]
                gp_name = gp_slug.replace('-', ' ').title()
                
                if gp_name:
                    year_races.append((year, round_num, gp_name))
                    seen_rounds.add(round_num)
            
            year_races.sort(key=lambda x: x[1])
            all_races.extend(year_races)
            print(f'  Found {len(year_races)} races')
            
        except Exception as e:
            print(f'Error on {year}: {e}')
        finally:
            driver.quit()
        time.sleep(2)
    
    return all_races

In [136]:
# Load from JSON cache (skip scraping if files exist)
import os
import json

def load_or_scrape(filename, years, scrape_func):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            data = [tuple(x) for x in json.load(f)]
        print(f'Loaded {len(data)} races from {filename}')
        return data
    else:
        data = scrape_func(years)
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)
        print(f'Scraped and saved {len(data)} races to {filename}')
        return data

### Scrape CYCLE 1

In [137]:
races_cycle1 = scrape_f1_races(cycle1)
print(f'Total: {len(races_cycle1)} races')
races_cycle1[:10]

Scraping 2017...
  Found 0 races
Scraping 2018...
  Found 21 races
Scraping 2019...
  Found 21 races
Scraping 2020...
  Found 17 races
Scraping 2021...
  Found 22 races
Total: 81 races


[(2018, 1, 'Australia'),
 (2018, 2, 'Bahrain'),
 (2018, 3, 'China'),
 (2018, 4, 'Azerbaijan'),
 (2018, 5, 'Spain'),
 (2018, 6, 'Monaco'),
 (2018, 7, 'Canada'),
 (2018, 8, 'France'),
 (2018, 9, 'Austria'),
 (2018, 10, 'Great Britain')]

In [138]:
# Save Cycle 1 to JSON
import json
with open('races_cycle1.json', 'w', encoding='utf-8') as f:
    json.dump(races_cycle1, f, indent=2)
print(f'Saved {len(races_cycle1)} races to races_cycle1.json')

Saved 81 races to races_cycle1.json


### Scrape CYCLE 2

In [139]:
races_cycle2 = scrape_f1_races(cycle2)
print(f'Total: {len(races_cycle2)} races')
races_cycle2[:10]

Scraping 2022...
  Found 22 races
Scraping 2023...
  Found 23 races
Scraping 2024...
  Found 24 races
Scraping 2025...
  Found 24 races
Total: 93 races


[(2022, 1, 'Bahrain'),
 (2022, 2, 'Saudi Arabia'),
 (2022, 3, 'Australia'),
 (2022, 4, 'Emiliaromagna'),
 (2022, 5, 'Miami'),
 (2022, 6, 'Spain'),
 (2022, 7, 'Monaco'),
 (2022, 8, 'Azerbaijan'),
 (2022, 9, 'Canada'),
 (2022, 10, 'Great Britain')]

In [140]:
# Save Cycle 2 to JSON
with open('races_cycle2.json', 'w', encoding='utf-8') as f:
    json.dump(races_cycle2, f, indent=2)
print(f'Saved {len(races_cycle2)} races to races_cycle2.json')

Saved 93 races to races_cycle2.json


### FastF1 Usage

In [141]:
# Example
year, rnd, gp = races_cycle2[0]
print(f'{gp} {year}, Round {rnd}')
print(f'ff1.get_session({year}, {rnd}, "R")')

Bahrain 2022, Round 1
ff1.get_session(2022, 1, "R")


### Collect Fastest Laps

In [142]:
# FastF1 function for historical data (2017-2022)
def format_laptime(td):
    """Format timedelta to M:SS.mmm"""
    if pd.isna(td):
        return None
    total_sec = td.total_seconds()
    minutes = int(total_sec // 60)
    seconds = total_sec % 60
    return f'{minutes}:{seconds:06.3f}'

def collect_fastest_laps_fastf1(races, save_file='temp_fastf1.csv'):
    """
    Collect fastest laps using FastF1.
    races: list of (year, round, gp_name) tuples
    """
    results = []
    
    for year, rnd, gp in races:
        print(f'{year} R{rnd} {gp}...', end=' ')
        try:
            session = ff1.get_session(year, rnd, 'R')
            session.load(laps=True, telemetry=False, weather=False, messages=False)
            
            if session.laps is not None and len(session.laps) > 0:
                fastest = session.laps.pick_fastest()
                if fastest is not None and len(fastest) > 0:
                    lap_time = fastest['LapTime']
                    formatted = format_laptime(lap_time)
                    results.append({
                        'Year': year,
                        'Round': rnd,
                        'GP': gp,
                        'Driver': fastest['Driver'],
                        'Team': fastest['Team'],
                        'LapTime': formatted,
                        'LapTime_sec': lap_time.total_seconds() if pd.notna(lap_time) else None
                    })
                    print(f"{fastest['Driver']} - {formatted}")
                else:
                    print('No fastest lap')
                    results.append({'Year': year, 'Round': rnd, 'GP': gp, 'Driver': None, 'Team': None, 'LapTime': None, 'LapTime_sec': None})
            else:
                print('No laps data')
                results.append({'Year': year, 'Round': rnd, 'GP': gp, 'Driver': None, 'Team': None, 'LapTime': None, 'LapTime_sec': None})
        except Exception as e:
            print(f'Error: {e}')
            results.append({'Year': year, 'Round': rnd, 'GP': gp, 'Driver': None, 'Team': None, 'LapTime': None, 'LapTime_sec': None})
        
        # Save progress
        pd.DataFrame(results).to_csv(save_file, index=False)
    
    return pd.DataFrame(results)

print('FastF1 collect function ready')

FastF1 collect function ready


In [143]:
# OpenF1 API with rate limit protection
import requests
import time
import os

def format_laptime_sec(seconds):
    """Format seconds to M:SS.mmm"""
    if seconds is None:
        return None
    minutes = int(seconds // 60)
    secs = seconds % 60
    return f'{minutes}:{secs:06.3f}'

def safe_request(url, max_retries=3, delay=1.0):
    """Request with retry and rate limit handling"""
    for attempt in range(max_retries):
        try:
            time.sleep(delay)
            r = requests.get(url, timeout=30)
            if r.status_code == 200:
                return r.json()
            elif r.status_code == 429:
                wait = 2 ** attempt * 10
                print(f'Rate limited, waiting {wait}s...')
                time.sleep(wait)
            else:
                return None
        except Exception as e:
            print(f'Request failed: {e}')
            time.sleep(5)
    return None

def get_fastest_laps_openf1(years, save_file='temp_fastest.csv'):
    """Get fastest laps using OpenF1 API (2023+ only)"""
    results = []
    
    for year in years:
        print(f'Fetching {year}...')
        sessions = safe_request(f'https://api.openf1.org/v1/sessions?year={year}&session_type=Race')
        
        if not sessions:
            print(f'  No data for {year}')
            continue
        
        for s in sessions:
            session_key = s['session_key']
            gp = s['country_name']
            print(f'  {gp}...', end=' ')
            
            laps = safe_request(f'https://api.openf1.org/v1/laps?session_key={session_key}&is_pit_out_lap=false')
            
            if laps:
                valid = [l for l in laps if l.get('lap_duration')]
                if valid:
                    fastest = min(valid, key=lambda x: x['lap_duration'])
                    lap_sec = fastest['lap_duration']
                    formatted = format_laptime_sec(lap_sec)
                    results.append({
                        'Year': year,
                        'GP': gp,
                        'DriverNum': fastest['driver_number'],
                        'LapTime': formatted,
                        'LapTime_sec': lap_sec
                    })
                    print(f"#{fastest['driver_number']} - {formatted}")
                else:
                    print('No valid laps')
            else:
                print('No data')
            
            pd.DataFrame(results).to_csv(save_file, index=False)
    
    return pd.DataFrame(results)

print('OpenF1 API ready (2023+ data)')

OpenF1 API ready (2023+ data)


In [144]:
# Fastest laps Cycle 1 (2017-2021) using FastF1
df_fastest1 = collect_fastest_laps_fastf1(races_cycle1, 'fastest_laps_cycle1.csv')
print(f'\nTotal: {len(df_fastest1)} races')
# Filter out failed entries for clean view
df_clean1 = df_fastest1[df_fastest1['Driver'].notna()]
print(f'Valid: {len(df_clean1)} races')
df_fastest1

2018 R1 Australia... 

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
core        WARNING 	Driver 5 completed the race distance 00:00.123000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cach

RIC - 1:25.945
2018 R2 Bahrain... 

core           INFO 	Finished loading data for 20 drivers: ['5', '77', '44', '10', '20', '27', '14', '2', '9', '31', '55', '16', '8', '18', '35', '11', '28', '7', '33', '3']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '3'
core        WARNING 	Fixed incorrect tyre stint information for driver '33'
core        WARNING 	Fixed incorrect tyre stint information for driver '27'


BOT - 1:33.740
2018 R3 China... 

...

core        WARNING 	Driver 44 completed the race distance 00:00.180000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '31', '3', '10', '16', '55', '99', '4', '18', '6', '14', '22', '7', '5', '11', '9', '63', '47']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


HAM - 1:30.734
2021 R22 United Arab Emirates... 

core        WARNING 	No lap data for driver 9
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 9)
core        WARNING 	Driver 33 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '55', '22', '10', '77', '4', '14', '31', '16', '5', '3', '18', '47', '11', '6', '99', '63', '7', '9']


VER - 1:26.103

Total: 81 races
Valid: 80 races


,Year,Round,GP,Driver,Team,LapTime,LapTime_sec
0,2018,1,Australia,RIC,Red Bull Racing,1:25.945,85.945
1,2018,2,Bahrain,BOT,Mercedes,1:33.740,93.740
2,2018,3,China,RIC,Red Bull Racing,1:35.785,95.785
3,2018,4,Azerbaijan,BOT,Mercedes,1:45.149,105.149
4,2018,5,Spain,RIC,Red Bull Racing,1:18.441,78.441
...,...,...,...,...,...,...,...
76,2021,18,Mexico,BOT,Mercedes,1:17.774,77.774
77,2021,19,Brazil,PER,Red Bull Racing,1:11.010,71.010
78,2021,20,Qatar,VER,Red Bull Racing,1:23.196,83.196
79,2021,21,Saudi Arabia,HAM,Mercedes,1:30.734,90.734


In [145]:
df_fastest1

,Year,Round,GP,Driver,Team,LapTime,LapTime_sec
0,2018,1,Australia,RIC,Red Bull Racing,1:25.945,85.945
1,2018,2,Bahrain,BOT,Mercedes,1:33.740,93.740
2,2018,3,China,RIC,Red Bull Racing,1:35.785,95.785
3,2018,4,Azerbaijan,BOT,Mercedes,1:45.149,105.149
4,2018,5,Spain,RIC,Red Bull Racing,1:18.441,78.441
...,...,...,...,...,...,...,...
76,2021,18,Mexico,BOT,Mercedes,1:17.774,77.774
77,2021,19,Brazil,PER,Red Bull Racing,1:11.010,71.010
78,2021,20,Qatar,VER,Red Bull Racing,1:23.196,83.196
79,2021,21,Saudi Arabia,HAM,Mercedes,1:30.734,90.734


In [146]:
# Fastest laps Cycle 2 (2022-2025)
# Uses full range - OpenF1 will skip years without data
years_cycle2 = [2022, 2023, 2024, 2025]
df_fastest2 = get_fastest_laps_openf1(years_cycle2, 'fastest_laps_cycle2.csv')
print(f'\nTotal: {len(df_fastest2)} races saved')
df_fastest2

Fetching 2022...
  No data for 2022
Fetching 2023...
  Bahrain... #24 - 1:33.996
  Saudi Arabia... #1 - 1:31.906
  Australia... #11 - 1:20.235
  Azerbaijan... #11 - 1:43.616
  Azerbaijan... #63 - 1:43.370
  United States... #1 - 1:29.708
  Monaco... #44 - 1:15.650
  Spain... #1 - 1:16.330
  Canada... #11 - 1:14.481
  Austria... #27 - 1:10.180
  Austria... #1 - 1:07.012
  Great Britain... #1 - 1:30.275
  Hungary... #1 - 1:20.504
  Belgium... #1 - 1:58.943
  Belgium... #44 - 1:47.305
  Netherlands... #14 - 1:13.837
  Italy... #81 - 1:25.072
  Singapore... #44 - 1:35.867
  Japan... #1 - 1:34.183
  Qatar... #1 - 1:25.604
  Qatar... #1 - 1:24.319
  United States... #1 - 1:39.060
  United States... #22 - 1:38.139
  Mexico... #44 - 1:21.334
  Brazil... #63 - 1:14.422
  Brazil... #4 - 1:12.486
  United States... #81 - 1:35.490
  United Arab Emirates... #1 - 1:26.993
Fetching 2024...
  Bahrain... #1 - 1:32.608
  Saudi Arabia... #16 - 1:31.632
  Australia... #16 - 1:19.813
  Japan... #1 - 1:33.7

,Year,GP,DriverNum,LapTime,LapTime_sec
0,2023,Bahrain,24,1:33.996,93.996
1,2023,Saudi Arabia,1,1:31.906,91.906
2,2023,Australia,11,1:20.235,80.235
3,2023,Azerbaijan,11,1:43.616,103.616
4,2023,Azerbaijan,63,1:43.370,103.370
...,...,...,...,...,...
83,2025,Brazil,23,1:12.400,72.400
84,2025,United States,1,1:33.365,93.365
85,2025,Qatar,10,1:23.188,83.188
86,2025,Qatar,81,1:22.996,82.996
